In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import zipfile
import glob
import shutil
import sqlite3

In [2]:
# Assigned variables to the paths
zip_path = 'Resources/reviews_all.zip'
unzipped = 'Resources/unzip-reviews'

# Use zipfile to extract the archive
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzipped)

In [3]:
# Assign the file names to a variable
review_files = ['reviews_0-250.csv','reviews_250-500.csv','reviews_500-750.csv','reviews_750-1250.csv','reviews_1250-end.csv']

# Use list comprehension to read the csv from the unzipped folder
df = [pd.read_csv(f'{unzipped}/{review_file}', low_memory=False) for review_file in review_files]
df = [pd.read_csv(file, low_memory=False) for file in glob.glob(f'{unzipped}/*.csv')]

In [ ]:
# Concatenate dataframes
merged_df = pd.concat(df, ignore_index=True)

# Delete the unzipped folder to avoid pushing to github
shutil.rmtree(unzipped)

merged_df.head()

In [5]:
# Merged on product id so the primary categories are assigned to each review, along with the ingredients if we decide to use them
product_info_df = pd.read_csv('Resources/product_info.csv', low_memory=False)

In [6]:
complete_df = pd.merge(merged_df, product_info_df[['product_id', 'secondary_category', 'ingredients']], on='product_id', how='left')

In [7]:
complete_df['submission_time'] = pd.to_datetime(complete_df['submission_time'], format='%Y-%m-%d')

In [ ]:
columns = ['product_id', 'product_name', 'brand_name', 'price_usd', 'secondary_category', 'rating', 'is_recommended', 'helpfulness', 'total_feedback_count', 'total_neg_feedback_count', 'total_pos_feedback_count', 'review_text', 'review_title', 'submission_time']
complete_df = complete_df[columns]
complete_df.head()

In [ ]:
complete_df.info()

In [ ]:
# Remove brands not related to the study
target_brands = ['Tatcha', 'The Ordinary', 'Dior', 'Estée Lauder', 'LANEIGE']

target_brands_df = complete_df[complete_df['brand_name'].isin(target_brands)]
count_by_target_brand = target_brands_df['brand_name'].value_counts()
count_by_target_brand

In [ ]:
product = target_brands_df[target_brands_df['product_id'] == 'P417604']
product

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='brand_name', y='rating', data=complete_df)
plt.xticks(rotation=45)
plt.show()

In [ ]:
avg_price_per_brand = target_brands_df.groupby('brand_name')['price_usd'].mean()
avg_price_per_brand.plot(kind='bar')

In [ ]:
top_products = target_brands_df.groupby('product_name')['total_feedback_count'].sum().sort_values(ascending=False).head(10)
top_products.plot(kind='bar')

In [ ]:
target_brands_df.groupby('brand_name')[['total_pos_feedback_count', 'total_neg_feedback_count']].sum().plot(kind='bar', stacked=True)

In [ ]:
brand_popularity = complete_df['brand_name'].value_counts().head(10)
sns.barplot(x=brand_popularity.values, y=brand_popularity.index)
plt.xlabel('Number of Reviews')
plt.ylabel('Brand Name')
plt.title('Top 10 Most Popular Brands')
plt.show()

In [ ]:
product_popularity = complete_df['product_name'].value_counts().head(10)
sns.barplot(x=product_popularity.values, y=product_popularity.index)
plt.xlabel('Number of Reviews')
plt.ylabel('Product Name')
plt.title('Top 10 Most Popular Products')
plt.show()

In [18]:
## This adds a new row that puts the brand name after the product name for display purposes,
## but also adds it to the dataframe, so don't run the last cell to output to csv if you run this cell
# complete_df['product_with_brand'] = complete_df['product_name'] + ' (' + complete_df['brand_name'] + ')'

# product_popularity = complete_df['product_with_brand'].value_counts().head(10)
# sns.barplot(x=product_popularity.values, y=product_popularity.index)
# plt.xlabel('Number of Reviews')
# plt.ylabel('Product (Brand)')
# plt.title('Top 10 Most Popular Products (with Brand Names)')
# plt.show()


In [ ]:
product_stats = complete_df.groupby('product_name').agg(
    total_reviews=pd.NamedAgg(column='is_recommended', aggfunc='count'),
    recommended_sum=pd.NamedAgg(column='is_recommended', aggfunc='sum'))
product_stats['recommended_percentage'] = (product_stats['recommended_sum'] / product_stats['total_reviews']) * 100
threshold = 5000
filtered_products = product_stats[product_stats['total_reviews'] >= threshold]
top_products = filtered_products.sort_values('recommended_percentage', ascending=False).head(20)
sns.barplot(x=top_products['recommended_percentage'], y=top_products.index)
plt.xlabel('Percentage of Recommended Reviews')
plt.ylabel('Product Name')
plt.title('Top Products by Percentage of Recommended Reviews (Min ' + str(threshold) + ' Reviews)')
plt.show()


In [ ]:
complete_df['helpfulness'].value_counts().head(35)

In [21]:
# Don't push this file to the repository, it's too large
# This is for the postgreSQL database and must be done from a local repository
# Uncomment this if you want to save it for yourself
# complete_df.to_csv('Resources/complete_df.csv', index=False, encoding='utf=8')


In [22]:
# JSON File conversion (Run this to get the JSON File, Keep on Local and do not push to repository)
# Too large for our purposes and cannot load on html. Uncomment this if you want to save it for yourself
# complete_df.to_json('Resources/complete_json.json', orient='records')

In [23]:
# Smaller subset that queries entries from 2022-03-01 to 2023-03
oneyear_df = complete_df[complete_df['submission_time'] >= pd.Timestamp('2022-03-01')]
oneyear_df.to_json('Resources/oneyear_df.json', orient='records')

In [ ]:
product_popularity = oneyear_df['product_name'].value_counts().head(10)
sns.barplot(x=product_popularity.values, y=product_popularity.index)
plt.xlabel('Number of Reviews')
plt.ylabel('Product Name')
plt.title('Top 10 Most Popular Products')
plt.show()

In [ ]:
brand_popularity = oneyear_df['brand_name'].value_counts().head(10)
sns.barplot(x=brand_popularity.values, y=brand_popularity.index)
plt.xlabel('Number of Reviews')
plt.ylabel('Brand Name')
plt.title('Top 10 Most Popular Brands')
plt.show()

In [26]:
# SQLite database creation
# Clean headers to prep for transferring into db
oneyear_df.columns = oneyear_df.columns.str.strip()

# Connect to SQLite
conn = sqlite3.connect('Resources/reviews.sqlite')

# Load to SQLite
oneyear_df.to_sql('reviews', conn, if_exists='replace', index=True)

# Close the connection
conn.close()